# Federated Horizontal XGBoost for PCR prediction   

federated horizontal xgboost learning on tabular data with bagging collaboration. 

Before do the training, we need to setup NVFLARE

## Setup NVFLARE

Follow [Getting Started](../../../../getting_started/readme.ipynb) to set up a virtual environment and install NVFLARE."



## Install requirements
assuming the current directory is 'home/yaqeen/Explainableand-Fair-Federated-Learning-with-XGBoost-for-Predicting-Pathological-Complete-Response-/xgboost'

In [1]:
!pwd

/home/yaqeen/Explainableand-Fair-Federated-Learning-with-XGBoost-for-Predicting-Pathological-Complete-Response-/training/xgboost_pcr


In [2]:
%pip install -r requirements.txt

  Using cached https://s3-us-west-2.amazonaws.com/xgboost-nightly-builds/federated-secure/xgboost-2.2.0.dev0%2B4601688195708f7c31fcceeb0e0ac735e7311e61-py3-none-manylinux_2_28_x86_64.whl (154.1 MB)

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


>Note:
In the upcoming sections, we'll utilize the 'tree' command. To install this command on a Linux system, you can use the sudo apt install tree command. As an alternative to 'tree', you can use the ls -al command.



## Prepare data
[prepare_data_pcr](../prepare_data.ipynb) notebooks. Pay attention to the current location. You need to switch directory to run the data split.
    

Now we have our data prepared. we are ready to do the training

### Data Cleaning 

so we need to do some clean up or skip certain rows. 
For example: certain floating number mistakenly add an alphabetical letter at some point of time. This may have already fixed by UCI. 


## XGBoost
This tutorial uses [XGBoost](https://github.com/dmlc/xgboost), which is an optimized distributed gradient boosting library.

### Federated XGBoost Model
Here we use **tree-based** collaboration for horizontal federated XGBoost.

Under this setting, individual trees are independently trained on each client's local data without aggregating the global sample gradient histogram information.
Trained trees are collected and passed to the server / other clients for bagging aggregation and further boosting rounds.

The XGBoost Booster api is leveraged to create in-memory Booster objects that persist across rounds to cache predictions from trees added in previous rounds and retain other data structures needed for training.

Let's look at the code see how we convert the local training script to the federated training script.

In [3]:
!pwd

/home/yaqeen/Explainableand-Fair-Federated-Learning-with-XGBoost-for-Predicting-Pathological-Complete-Response-/training/xgboost_pcr


In [4]:
!cat code/xgboost_fl.py

# Copyright (c) 2023, NVIDIA CORPORATION.  All rights reserved.
# Licensed under the Apache License, Version 2.0

import argparse
import csv
import json
from typing import Dict, List, Tuple
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from nvflare import client as flare
from nvflare.app_opt.xgboost.tree_based.shareable_generator import update_model


def to_dataset_tuple(data: dict):
    dataset_tuples = {}
    for dataset_name, dataset in data.items():
        dataset_tuples[dataset_name] = _to_data_tuple(dataset)
    return dataset_tuples

def _to_data_tuple(data):
    data_num = data.shape[0]
    x = data.iloc[:, 1:]
    y = data.iloc[:, 0]
    return x.to_numpy(), y.to_numpy(), data_num

def load_features(feature_data_path: str) -> List:
    try:
        with open(feature_data_path, "r") as file:
  

The code is pretty much like the local xgboost training script of `code/xgboost_local_oneshot.py`, and its derivative `code/xgboost_local_iter.py`.

#### load data

We first load the features from the header file: 
    
```
    site_name = flare.get_site_name()
    feature_data_path = f"{data_root_dir}/{site_name}_header.csv"
    features = load_features(feature_data_path)
    n_features = len(features) -1

    data_path = f"{data_root_dir}/{site_name}.csv"
    data = load_data(data_path=data_path, data_features=features, test_size=test_size, skip_rows=skip_rows)

```

then load the data from the main csv file, then transform the data and split the training and test data based on the test_size provided.  

```
    data = to_dataset_tuple(data)
    dataset = transform_data(data)
    x_train, y_train, train_size = dataset["train"]
    x_test, y_test, test_size = dataset["test"]

```

The part that's specific to Federated Learning is in the following codes

```
# (1) import nvflare client API
from nvflare import client as flare

```
```
# (2) initializes NVFlare client API
    flare.init()

    site_name = flare.get_site_name()
    
```
    
These few lines, import NVFLARE Client API and initialize it, then use the API to find the site_name (such as site-1, site-2 etc.). With the site-name, we can construct the site-specific 
data path such as

```
    feature_data_path = f"{data_root_dir}/{site_name}_header.csv"

    data_path = f"{data_root_dir}/{site_name}.csv"
```

#### Training 

In the standard traditional xgboost, we would train the model such as
```
  model = xgb.train(...) 
```

with federated learning, using FLARE Client API, we need to make a few changes
* 1) we are not only training in local iterations, but also global rounds, we need to keep the program running until we reached to the totoal number of rounds 
  
  ```
      while flare.is_running():
          ... rest of code
  
  ```
  
* 2) Unlike local learning, we have now have more than one clients/sites participating the training. To ensure every site starts with the same model parameters, we use server to broadcast the initial model parameters to every sites at the first round ( current_round = 0). 

* 3) We will need to use FLARE client API to receive global model updates 

```
        # (3) receives FLModel from NVFlare
        input_model = flare.receive()
        global_params = input_model.params
        curr_round = input_model.current_round
```

```
        if curr_round == 0:
            # (4) first round, no global model
            model = xgb.train(
                xgb_params,
                dmat_train,
                num_boost_round=1,
                evals=[(dmat_train, "train"), (dmat_test, "test")],
            )
            config = model.save_config()
        ....
```
* 4) if it is not the first round, we need to use the global model to update the local model before training the next round. 

```
            # (5) update model based on global updates
            model_updates = global_params["model_data"]
            for update in model_updates:
                global_model_as_dict = update_model(global_model_as_dict, json.loads(update))
            loadable_model = bytearray(json.dumps(global_model_as_dict), "utf-8")
            # load model
            model.load_model(loadable_model)
            model.load_config(config)
```

* 5) we then evaluate the global model using the local data

```
            # (6) evaluate model
            auc = evaluate_model(x_test, model, y_test)
```
* 6) finally we do the training 

```
            # train model in two steps
            # first, eval on train and test
            eval_results = model.eval_set(
                evals=[(dmat_train, "train"), (dmat_test, "test")], iteration=model.num_boosted_rounds() - 1
            )
            print(eval_results)
            # second, train for one round
            model.update(dmat_train, model.num_boosted_rounds())
        
```

* 7) we need the new training result (new tree) back to server for aggregation, to do that, we have the following code

```
        # (7) construct trained FL model
        # Extract newly added tree using xgboost_bagging slicing api
        bst_new = model[model.num_boosted_rounds() - 1 : model.num_boosted_rounds()]
        local_model_update = bst_new.save_raw("json")
        params = {"model_data": local_model_update}
        metrics = {"accuracy": auc}
        output_model = flare.FLModel(params=params, metrics=metrics)

        # (8) send model back to NVFlare
        flare.send(output_model)
```

## Prepare Job  

Now, we have the code, we need to prepare job folder with configurations to run in NVFLARE. To do this, We are going to use Job API to construct a FedJob, which can be used directly run simulation and export job config. 

In [2]:
! cat xgboost_htree_job.py

# Copyright (c) 2024, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
from nvflare import FedJob
from nvflare.app_common.workflows.scatter_and_gather import ScatterAndGather
from nvflare.app_opt.xgboost.tree_based.bagging_aggregator import XGBBaggingAggregator
from nvflare.app_opt.xgboost.tree_based.model_persistor import XGBModelPersistor
from nvflare.app_opt.xgboost.tree_based.shareable_generator import XGBModelShareableGenerator
from nvflare.job_config.

>Note 
 For potential on-the-fly data cleaning, we use skip_rows = 0 to skip 1st row. We could skip_rows = [0, 3] to skip first and 4th rows.



## Run job in simulator

We use the simulator to run this job.  There are two ways: 

**Run FedJob.simulator_run()**

``` python xgboost_htree_job.py ```

In [35]:
! python xgboost_htree_job.py 

2025-07-24 11:55:36,907 - SimulatorRunner - INFO - Create the Simulator Server.
2025-07-24 11:55:36,908 - CoreCell - INFO - server: creating listener on tcp://0:44881
2025-07-24 11:55:36,933 - CoreCell - INFO - server: created backbone external listener for tcp://0:44881
2025-07-24 11:55:36,933 - ConnectorManager - INFO - 111319: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2025-07-24 11:55:36,933 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:12088] is starting
2025-07-24 11:55:37,434 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:12088
2025-07-24 11:55:37,434 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:44881] is starting
2025-07-24 11:55:37,436 - SimulatorServer - INFO - max_reg_duration=60.0
2025-07-24 11:55:37,546 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 39619
2025-07-24 11:55:37,546 - SimulatorRunner - INFO - 

**Run Simuator CLI with job config folder**

```
!nvflare simulator /tmp/nvflare/jobs/xgboost -w /tmp/nvflare/xgboost -n 3 -t 3
```

Let's examine the results.

We can notice from the FL training log, at the last round (`current_round=99`) of local training, site-1 reports `site-1: global model AUC: 0.82085`
Under tree-based collaboration, the newly boosted trees from all clients will be bagged together to be added to the local models, hence, at the end of our 100-round training with 3 clients, the global model should have 300 trees in total. Let's double check if this holds:

In [4]:
ls -al ./xgboost/server/simulate_job/app_server/xgboost_model.json

-rw-rw-r-- 1 yaqeen yaqeen 13405 Jul 24 11:05 ./xgboost/server/simulate_job/app_server/xgboost_model.json


In [5]:
import json

# Specify the global model file path
file_path =  './xgboost/server/simulate_job/app_server/xgboost_model.json'

with open(file_path) as json_data:
    model = json.load(json_data)
    print(f"num_trees: {model['learner']['gradient_booster']['model']['gbtree_model_param']['num_trees']}")

num_trees: 12


And yes, we do have 300 trees for the global model as expected.

Now let's run some local trainings to verify if this number makes sense.

In [6]:
!python3 ./code/xgboost_local_oneshot.py --data_root_dir /home//yaqeen/NVFlare/dataset/pcr/output

[0]	train-auc:0.85264	test-auc:0.58625
[1]	train-auc:0.88581	test-auc:0.56751
[2]	train-auc:0.88896	test-auc:0.57455
local model AUC: 0.57455


In [4]:
!python3 ./code/xgboost_local_iter_01.py --data_root_dir /home/yaqeen/NVFlare/dataset/pcr/output


📍 Training for: site-1
🔁 site-1 | Round 0
[0]	train-auc:0.81005	test-auc:0.54135
📊 site-1 | AUC=0.54135, Balanced Accuracy=0.50000
✅ site-1 | 🔥 New best model saved: AUC=0.54135, BalAcc=0.50000
🔁 site-1 | Round 1
[0]	train-auc:0.81004901960784315	test-auc:0.54135338345864659
📊 site-1 | AUC=0.57895, Balanced Accuracy=0.50000
🔁 site-1 | Round 2
[1]	train-auc:0.81474673202614378	test-auc:0.57894736842105265
📊 site-1 | AUC=0.56391, Balanced Accuracy=0.48684
🔁 site-1 | Round 3
[2]	train-auc:0.86754493464052285	test-auc:0.56390977443609025
📊 site-1 | AUC=0.54699, Balanced Accuracy=0.62218
✅ site-1 | 🔥 New best model saved: AUC=0.54699, BalAcc=0.62218
🔁 site-1 | Round 4
[3]	train-auc:0.90328839869281041	test-auc:0.54699248120300747
📊 site-1 | AUC=0.54887, Balanced Accuracy=0.63534
✅ site-1 | 🔥 New best model saved: AUC=0.54887, BalAcc=0.63534
🔁 site-1 | Round 5
[4]	train-auc:0.91176470588235292	test-auc:0.54887218045112784
📊 site-1 | AUC=0.55451, Balanced Accuracy=0.63534
🔁 site-1 | Round 6


In [8]:
%env DATASET_ROOT=/home/yaqeen/NVFlare/dataset/pcr/xgboost

env: DATASET_ROOT=/home/yaqeen/NVFlare/dataset/pcr/xgboost


In [37]:
!python3 code/model_validation.py \
        --data_path "${DATASET_ROOT}/test_selected_features.csv" \
        --model_path "./xgboost/server/simulate_job/app_server/xgboost_model.json" \
        --size_valid 306 --num_trees 300

AUC over first 306 instances is: 0.6551819879852592
Balanced Accuracy over first 306 instances is: 0.6576
